In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

import time

from sklearn.metrics import mean_absolute_error

# Dynamic Pricing with Reinforcement Learning

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/dynamic_pricing/data/preprocessed_data_v3.csv")
df

,DWHotelID,Unnamed: 0,w,Date,ListPriceTRY,PurchaseAmountTRY,RatingTripA,ProfitRate,featureScore,Date_year,Date_week,rooms_old,#rooms,empty_rooms,rewardrate
0,58.0,0,10.0,2013-21,486.200000,407.300000,4.5,0.193715,2.253854,2013,21,2.0,379.0,369.001,0.213821
1,58.0,1,12.0,2013-22,805.833333,639.416667,4.5,0.260263,2.253854,2013,22,2.0,379.0,367.001,0.453450
2,58.0,2,31.0,2013-23,863.870968,691.387097,4.5,0.249475,2.253854,2013,23,2.0,379.0,348.001,0.495642
3,58.0,3,58.0,2013-24,643.258621,516.724138,4.5,0.244878,2.253854,2013,24,2.0,379.0,321.001,0.394187
4,58.0,4,50.0,2013-25,1223.840000,988.900000,4.5,0.237577,2.253854,2013,25,2.0,379.0,329.001,0.714101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10666,390.0,11515,0.0,2021-34,2099.000000,1805.000000,4.0,0.162881,0.607659,2021,34,17.0,119.0,119.001,2.470567
10667,390.0,11516,0.0,2021-35,2099.000000,1805.000000,4.0,0.162881,0.607659,2021,35,17.0,119.0,119.001,2.470567
10668,390.0,11517,0.0,2021-36,2099.000000,1805.000000,4.0,0.162881,0.607659,2021,36,17.0,119.0,119.001,2.470567
10669,390.0,11518,0.0,2021-37,2099.000000,1805.000000,4.0,0.162881,0.607659,2021,37,17.0,119.0,119.001,2.470567


In [4]:
print('Max Purchase Amount:', df['PurchaseAmountTRY'].max() , 'Min Purchase Amount:', df['PurchaseAmountTRY'].min())
print('Max List Price:', df['ListPriceTRY'].max() , 'Min List Price:', df['ListPriceTRY'].min() )
print('Max Rate:', df['RatingTripA'].max(), 'Min Rate:', df['RatingTripA'].min())
print('Max Feature Score:', df['featureScore'].max() , 'Min Feature Score:', df['featureScore'].min())
print('Max Room Count:', df['empty_rooms'].max() , 'Min Room Count:', df['empty_rooms'].min())

Max Purchase Amount: 3671.2 Min Purchase Amount: 34.0
Max List Price: 4000.0 Min List Price: 37.66666666666666
Max Rate: 5.0 Min Rate: 2.5
Max Feature Score: 2.2538542161050463 Min Feature Score: 0.469892026578073
Max Room Count: 492.001 Min Room Count: 0.001


In [5]:
# Utility function for creating discreate space
def round_specific(x, precision, base):
  return round(base * round(float(x)/base),precision)

In [6]:
class Sale:
    def __init__(self, purchase, customer_traffic, competitiveness, empty_rooms):
        self.profit = round_specific(np.random.rand(), 2, 0.01)
        self.purchase = purchase
        if self.purchase > 4000:
            self.purchase = 4000
        if self.purchase < 0:
            self.purchase = 0
        
        self.list_price = self.purchase + self.purchase*self.profit
        self.list_price = round_specific(self.list_price, 0, 50)
        if self.list_price > 4000:
            self.list_price = 4000
        if self.list_price < 0:
            self.list_price = 0

        self.customer_traffic = customer_traffic
        self.competitiveness = competitiveness
        self.empty_rooms = empty_rooms
    
    def __str__(self):
        return f"{self.profit}"
    
    def __sub__(self,other):
        return self.profit - other.profit
    
    def action(self, choice):
        if choice == 0:
            self.change(-5) # profiti %5 indir
        elif choice == 1:
            self.change(0) 
        elif choice == 2:
            self.change(+10) 
        """
        elif choice == 3:
            self.change(+5) 
        elif choice == 4:
            self.change(+10) # profiti %10 arttir 
        """
    
    def change(self, p=False):
        if not p:
            self.profit = round_specific(np.random.rand(), 2, 0.01)
        else:
            self.profit = self.profit + (p/100)
            
        self.list_price = self.purchase + self.purchase*self.profit 
        self.list_price = round_specific(self.list_price, 0, 50)
        if self.list_price > 4000:
            self.list_price = 4000
        if self.list_price < 0:
            self.list_price = 0

In [ ]:
q_table_main = np.load('/content/drive/My Drive/Colab Notebooks/dynamic_pricing/data/qtable/q_table_50_4050_3acts.npy',allow_pickle='TRUE').item()

In [ ]:
def RL(HM_EPISODES, epsilon, df, q_table):
    EPS_DECAY = 0.998
    SHOW_EVERY = 100
    LEARNING_RATE = 0.1
    DISCOUNT = 0.95
    df['list_price']=np.nan

    episode_rewards = []

    for episode in range(HM_EPISODES):
    
        sale = df.iloc[episode]
        
        purchase = sale['PurchaseAmountTRY']
        purchase = round_specific(purchase, 0, 50)
        
        customer_traffic = sale['RatingTripA']
        customer_traffic = round_specific(customer_traffic, 1, 0.0625)
        
        competitiveness = sale['featureScore']
        competitiveness = round_specific(competitiveness, 2, 0.03125)
        
        empty_rooms = sale['empty_rooms']
        
        agent = Sale(purchase, customer_traffic, competitiveness, empty_rooms)
        
        list_price = agent.list_price
        """
        if episode % SHOW_EVERY == 0:
            print(f"on #{episode}, epsilon is {epsilon}")
            print(f"{SHOW_EVERY} ep mean: {np.mean(episode_rewards[-SHOW_EVERY:])}")
            show = True
        else:
            show = False
        """
        episode_reward = 0
        for i in range(200):
            obs = (purchase, list_price, customer_traffic, competitiveness)
            
            if np.random.random() > epsilon:
                # GET THE ACTION
                action = np.argmax(q_table[obs])
            else:
                action = np.random.randint(0, 3)
            
            # Take the action!
            agent.action(action)
            
            ##############################
            # REWARD FUNCTION
            new_obs = (agent.purchase, agent.list_price, agent.customer_traffic, agent.competitiveness)
            max_future_q = np.max(q_table[new_obs])
            current_q = q_table[obs][action]
            
            rewardrate = (agent.list_price-agent.purchase)/ agent.empty_rooms
            
            if rewardrate < 1:
                reward = -300
                new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            elif rewardrate < 10 or rewardrate > 1000:
                new_q = rewardrate
                reward = 25
            else:
                reward = -1
                new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            
            q_table[obs][action] = new_q

            episode_reward += reward
            if reward ==25:
                break
        df['list_price'].iloc[episode] = agent.list_price
        #print(episode_reward)
        episode_rewards.append(episode_reward)
        epsilon *= EPS_DECAY

    moving_avg = np.convolve(episode_rewards, np.ones((SHOW_EVERY,))/SHOW_EVERY, mode='valid')
    return df, moving_avg

**Hotel Based Pricing**

In [ ]:
df_1M = df


df_list = [df_1M]

In [ ]:
for data_frame in df_list:
  q_table = q_table_main
  data_frame.reset_index()
  data_frame, moving_avg = RL(data_frame.shape[0], 0.4, data_frame, q_table)
  
  plt.rcParams['figure.figsize'] = [20, 5]
  plt.plot([i for i in range(len(moving_avg))], moving_avg)
  plt.ylabel(f"Reward 100ma")
  plt.xlabel("episode #")
  plt.show()

In [ ]:
df_1M2 = df_1M[['ListPriceTRY','list_price']]
plt.figure();
plt.rcParams['figure.figsize'] = [20, 5]
df_1M2.plot.area(stacked=False);

**Original List Price Classfication**

In [ ]:
# Utility function for creating discreate space
def round_specific_series(x, precision, base):
  return round(base * round(x/base),precision)

In [ ]:
for x in df_list:
    x["ClassifiedListPriceTRY"] = round_specific_series(x["ListPriceTRY"], 0, 100)

**Mean Absolute Percentage Error**

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

**Print Metrics**

In [ ]:
def print_metrics(data_frame):

  print("*Number of Sales: ",data_frame.shape[0])
  print("*Mean Absolute Error: ", mean_absolute_error(data_frame['ListPriceTRY'],data_frame['list_price']));
  print("*Mean Absolute Error of Classified: ", mean_absolute_error(data_frame['ClassifiedListPriceTRY'],data_frame['list_price']));

  print("*h460 MAPE of Or-Re: ", mean_absolute_percentage_error(data_frame['ListPriceTRY'],data_frame['list_price']));
  print("*h460 MAPE of Cl-Re: ", mean_absolute_percentage_error(data_frame['ClassifiedListPriceTRY'],data_frame['list_price']));

  print("*Mean of Recommended List Prices:", data_frame['list_price'].mean())
  print("*Mean of Original List Prices:", data_frame['ListPriceTRY'].mean())
  print("*Mean of Classified List Prices:", data_frame['ClassifiedListPriceTRY'].mean())

  print("*Standard Deviation of Recommended List Prices:", data_frame['list_price'].std())
  print("*Standard Deviation of Original List Prices:", data_frame['ListPriceTRY'].std())
  print("*Standard Deviation of Classified List Prices:", data_frame['ClassifiedListPriceTRY'].std())
  
  return

In [ ]:
for d in df_list:
  print_metrics(d)
  print("|")